### Stock Database Analysis

In [2]:
from stocksense.database_handler import DatabaseHandler

db = DatabaseHandler()

2024-12-01 22:29:22.950 | SUCCESS  | stocksense.database_handler.schema:create_tables:121 - Tables created successfully


In [2]:
import polars as pl

from stocksense.pipeline.scraper import Scraper

base_df = pl.read_csv(
    "../stocksense/data/interim/SP500.csv", separator=";", columns=["tic", "name", "sector"]
)
curr_df = Scraper.scrape_sp500_constituents()
base_df

tic,sector,name
str,str,str
"""AABA""","""Financials""",null
"""AAP""","""Consumer Discretionary""",null
"""ABC""","""Health Care""",null
"""ABI""","""Industrials""",null
"""ABKFQ""","""Financials""",null
…,…,…
"""XYL""","""Industrials""","""Xylem Inc."""
"""YUM""","""Consumer Discretionary""","""Yum! Brands"""
"""ZBRA""","""Information Technology""","""Zebra Technologies"""


In [3]:
additions, removals = Scraper.scrape_sp500_changes()
additions

added,tic,name_added
date,str,str
2024-11-26,"""TPL""","""Texas Pacific Land Corporation"""
2024-09-30,"""AMTM""","""Amentum"""
2024-09-23,"""PLTR""","""Palantir Technologies"""
2024-09-23,"""DELL""","""Dell Technologies"""
2024-09-23,"""ERIE""","""Erie Indemnity"""
…,…,…
1999-06-09,"""WLP""","""Wellpoint"""
1998-12-11,"""FSR""","""Firstar"""
1998-12-11,"""CCL""","""Carnival Corp."""


In [4]:
df = base_df.join(additions, on="tic", how="left")
df = df.join(removals, on="tic", how="left")
df = df.join(curr_df.select(["tic", "date_added"]), on="tic", how="left")
df = df.with_columns(pl.col("removed").alias("date_removed"))
df = df.with_columns(
    pl.when(pl.col("date_added").is_null())
    .then(pl.col("added"))
    .otherwise(pl.col("date_added"))
    .alias("date_added")
).with_columns(
    pl.when(pl.col("name").is_null())
    .then(pl.col("name_removed"))
    .otherwise(pl.col("name"))
    .alias("name")
)
df = df.filter(~(pl.col("date_added").is_null() & pl.col("date_removed").is_null()))

df = df[["tic", "name", "sector", "date_added", "date_removed"]]

In [2]:
stocks = db.fetch_stock("CPAY")
stocks

tic,name,sector,date_added,date_removed
str,str,str,date,date
"""CPAY""","""Corpay""","""Financials""",2018-06-20,null


In [3]:
db.fetch_market_data("AMZN").sort(by=["date"])

tic,date,close,adj_close,volume
str,date,f64,f64,i64
"""AMZN""",2005-01-03,2.226,2.226,208930000
"""AMZN""",2005-01-04,2.107,2.107,388370000
"""AMZN""",2005-01-05,2.0885,2.0885,167084000
"""AMZN""",2005-01-06,2.0525,2.0525,174018000
"""AMZN""",2005-01-07,2.116,2.116,196732000
…,…,…,…,…
"""AMZN""",2024-11-22,197.119995,197.119995,31530800
"""AMZN""",2024-11-25,201.449997,201.449997,40685700
"""AMZN""",2024-11-26,207.860001,207.860001,41673700


In [7]:
db.fetch_insider_data("PLTR")

,tic,filling_date,trade_date,owner_name,title,transaction_type,last_price,qty,shares_held,owned,value
0,PLTR,2020-10-02,2020-09-30,Cohen Stephen Andrew,"Pres, Secretary",S - Sale+OE,$9.71,"-2,000,000","2,402,647",-45%,"-$19,422,100"
1,PLTR,2020-10-02,2020-09-30,Glazer David A.,"CFO, Treasurer",S - Sale,$10.03,"-1,615,621","3,940,152",-29%,"-$16,206,016"
2,PLTR,2020-10-02,2020-09-30,Karp Alexander C.,CEO,S - Sale+OE,$9.70,"-11,500,000","9,026,496",-56%,"-$111,606,220"
3,PLTR,2020-10-02,2020-09-30,Long Matthew A.,GC,S - Sale+OE,$10.06,"-579,404","874,006",-40%,"-$5,830,660"
4,PLTR,2020-10-02,2020-09-30,Moore Alexander D.,Dir,S - Sale,$9.54,"-285,000","2,429,249",-11%,"-$2,719,315"
...,...,...,...,...,...,...,...,...,...,...,...
277,PLTR,2024-10-07,2024-10-03,Planishek Heather A.,See Remarks,S - Sale+OE,$39.32,"-34,017","570,136",-6%,"-$1,337,500"
278,PLTR,2024-10-08,2024-10-04,Taylor Ryan D.,See Remarks,S - Sale+OE,$40.00,"-195,500","331,369",-37%,"-$7,820,704"
279,PLTR,2024-10-15,2024-10-11,Stat Lauren Elaina Friedman,Dir,S - Sale,$43.90,"-7,321","201,243",-4%,"-$321,392"
280,PLTR,2024-10-29,2024-10-25,Karp Alexander C.,See Remarks,S - Sale+OE,$45.01,"-5,656,293","6,432,258",-47%,"-$254,610,669"


In [4]:
db.fetch_financial_data("ZTS")

tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct
str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ZTS""",2010-12-31,2011-03-01,null,null,null,null,null,null,null,0.0,null,null,null,null,null,null,0.0,null,0.0,254.0,-9.0,-277.0,0.0,null,null,null
"""ZTS""",2011-12-31,2012-02-29,null,null,null,null,null,null,2311.0,5711.0,79.0,871.0,1063.0,2149.0,843.0,575.0,1975.0,null,3736.0,243.0,-440.0,247.0,0.0,null,null,null
"""ZTS""",2012-03-31,2012-05-30,1047.0,393.0,338.0,111.0,227.0,500.0,null,0.0,null,null,null,null,null,null,0.0,null,0.0,-4.0,-33.0,71.0,52.0,null,null,null
"""ZTS""",2012-06-30,2012-08-29,1094.0,378.0,344.0,173.0,314.0,500.0,null,0.0,null,null,null,null,null,null,0.0,null,0.0,74.0,-26.0,-53.0,10.0,null,null,null
"""ZTS""",2012-09-30,2012-11-29,1019.0,359.0,335.0,162.0,274.0,500.0,null,0.0,null,null,null,null,null,null,0.0,null,0.0,74.0,-30.0,-18.0,1.0,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZTS""",2023-09-30,2023-11-02,2151.0,638.0,525.0,596.0,942.0,459.525,6222.0,14106.0,1751.0,1257.0,2744.0,3318.0,1608.0,6552.0,9032.0,10140.0,5078.0,724.0,-271.0,-416.0,173.0,145.0,11632.0,0.001
"""ZTS""",2023-12-31,2024-02-13,2213.0,728.0,565.0,525.0,867.0,458.367,6343.0,14286.0,2039.0,1304.0,2564.0,3434.0,1889.0,6564.0,9295.0,10295.0,4997.0,897.0,-210.0,-397.0,172.0,198.0,11564.0,-0.06
"""ZTS""",2024-03-31,2024-04-29,2190.0,643.0,547.0,599.0,954.0,456.947,6359.0,14348.0,1973.0,1293.0,2651.0,3476.0,1909.0,6562.0,9296.0,10696.0,5058.0,595.0,-131.0,-524.0,198.0,140.0,11644.0,0.03


In [9]:
st = db.fetch_stock().to_pandas()
st

,tic,name,sector,date_added,date_removed
0,AAP,Advance Auto Parts,Consumer Discretionary,2015-07-08,2023-08-25
1,ABMD,Abiomed,Health Care,2018-05-31,2022-12-22
2,ABS,Albertsons,Consumer Staples,NaT,2006-06-02
3,ACAS,American Capital,Financials,NaT,2009-03-03
4,ADCT,ADC Telecommunications,Communication Services,NaT,2007-07-02
...,...,...,...,...,...
783,XYL,Xylem Inc.,Industrials,2011-11-01,NaT
784,YUM,Yum! Brands,Consumer Discretionary,1997-10-06,NaT
785,ZBRA,Zebra Technologies,Information Technology,2019-12-23,NaT
786,ZBH,Zimmer Biomet,Health Care,2001-08-07,NaT
